In [ ]:
import numpy as np
import pandas as pd
import yaml

In [311]:

with open('./config.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    data = yaml.load(file, Loader=yaml.FullLoader)

In [369]:
time = pd.to_datetime('today').normalize().Timestamp('2015-10-14 00:00:00')

AttributeError: 'Timestamp' object has no attribute 'Timestamp'

In [374]:
pd.Timestamp("today").strftime("%m/%d/%Y")

'03/03/2021'

In [312]:
data_sheets = data['FilePaths']

In [313]:
data_sheets

{'AGR': './data/data-sheet-agriculture.xlsx',
 'BLD': './data/data-sheet-buildings.xlsx',
 'IND': './data/data-sheet-industry.xlsx',
 'TRN': './data/data-sheet-transport.xlsx',
 'OWN': './data/data-sheet-ownuse.xlsx',
 'PIP': './data/data-sheet-pipeline transport.xlsx',
 'NON': './data/data-sheet-nonspecified.xlsx'}

#### Read in emissions data and mapping file and format

In [314]:
# Read in emissions data

In [315]:
IPCC_Emission = pd.read_excel(data['Maping_File'], sheet_name = "Emission Factors", header=0,index_col="Fuel type ")

In [316]:
# Read in maping sheet sheet

In [317]:
IPCC_map = pd.read_excel(data['Maping_File'] , sheet_name = "MAP", header=1,index_col="Fuel type")

In [318]:
#Select only the columns that are going to be used In the emissions file. 

In [319]:
IPCC_Emission = IPCC_Emission[["Default Carbon content(kg/GJ)"]]

In [320]:


IPCC_Emission.columns = ["Carbon content(kg/GJ)"]

In [321]:
#Join emissions data EGEDA maping file. 

In [322]:
Emission_egeda = IPCC_map.join(IPCC_Emission,on="Fuel type")

In [323]:
Emission_egeda = Emission_egeda.dropna()

In [324]:
Emission_egeda= Emission_egeda.set_index(['Fuel','Account'])

In [325]:
# Set biomass emissions to 0

In [326]:
Emission_egeda.loc[pd.IndexSlice[:,"LULUCF"],:] = 0

In [327]:
Emission_egeda = Emission_egeda.droplevel(level = 1, axis =0)

In [328]:
#Drop 2017 fuel names 

In [329]:
Emission_egeda = Emission_egeda.drop(2017, axis =1 )

In [330]:
#Convert From tonnes of carbon to tonnes of carbon dioxide

In [331]:
Emission_egeda = Emission_egeda*3.67/1000000

In [332]:
Emission_egeda.index = Emission_egeda.index.rename("FUEL")

In [333]:
#Set out column order for osemosys sheet

In [334]:
start_cols = ["SCENARIO",'REGION','TECHNOLOGY','EMISSION','MODE_OF_OPERATION','UNITS','NOTES']

In [335]:
end_cols =  [*range(2017,2071)]

In [336]:
all_cols = start_cols + end_cols

In [337]:
#Not being used currently

#This function generates a single emissions "co2" factor for each Technology. 

#def demand_emissions_by_tech_co2(df_data_sheet,Emission_map):
#    df_EmissionActivityRatio = df_data_sheet.merge(right = Emission_map, left_on= ["FUEL"], right_index=True)
#    df_EmissionActivityRatio.iloc[:,6:-1] = df_EmissionActivityRatio.iloc[:,6:-1].mul(df_EmissionActivityRatio.loc[:,"Carbon content(kg/GJ)"],axis =0 ) *1000
#    df_EmissionActivityRatio = df_EmissionActivityRatio.set_index("REGION","TECHNOLOGY","FUEL").groupby(["REGION","TECHNOLOGY"]).sum()
#    df_EmissionActivityRatio = df_EmissionActivityRatio.reset_index()
#    df_EmissionActivityRatio["MODE_OF_OPERATION"] = 1
#    df_EmissionActivityRatio["UNITS"] = 'Mt CO2 per PJ"
#    df_EmissionActivityRatio["NOTES"] = np.nan
#    df_EmissionActivityRatio["EMISSION"] = "CO2"
#    df_EmissionActivityRatio["SCENARIO"] = "Current"

#    return df_EmissionActivityRatio

In [338]:
#This function generates a emissions "co2" factor By fuel. 

def demand_emissions(df_data_sheet,Emission_map):
    df_EmissionActivityRatio = df_data_sheet.merge(right = Emission_map, left_on= ["FUEL"], right_index=True)
    df_EmissionActivityRatio.iloc[:,6:-1] = df_EmissionActivityRatio.iloc[:,6:-1].mul(df_EmissionActivityRatio.loc[:,"Carbon content(kg/GJ)"],axis =0 ) *1000
    #df_EmissionActivityRatio = df_EmissionActivityRatio.set_index("REGION","TECHNOLOGY","FUEL").groupby(["REGION","TECHNOLOGY"]).sum()
    #df_EmissionActivityRatio = df_EmissionActivityRatio.reset_index()
    df_EmissionActivityRatio["EMISSION"] = df_EmissionActivityRatio["FUEL"].astype(str) + "_CO2"
    df_EmissionActivityRatio = df_EmissionActivityRatio.drop(["FUEL","Carbon content(kg/GJ)"],axis =1 )
    df_EmissionActivityRatio["MODE_OF_OPERATION"] = 1
    df_EmissionActivityRatio["UNITS"] = "Mt CO2 per PJ"
    df_EmissionActivityRatio["NOTES"] = np.nan



    return df_EmissionActivityRatio

In [339]:
#Run through Data sheets applying the above function. If the scenario column is not present Add it and include emissions for both scenarios

In [340]:
df_emissions_activity = pd.DataFrame()

for paths in data_sheets:
    input_activity = pd.read_excel(data_sheets[paths], sheet_name = "InputActivityRatio", header=0)
    print ("reading " + f'{paths} at {data_sheets[paths]}')
    try: 
        EmissionActivityRatio = demand_emissions(input_activity,Emission_egeda)
        EmissionActivityRatio = EmissionActivityRatio[all_cols]
        #EmissionActivityRatio.to_excel(data['Output_location']+ "EmissionActivityRatio" + paths+".xlsx" ,index= False)
        print ( f' Saved to {data_sheets[paths]}')
    except:
        EmissionActivityRatio = demand_emissions(input_activity,Emission_egeda)
        EmissionActivityRatio["SCENARIO"] = "Net-zero"
        EmissionActivityRatio = EmissionActivityRatio[all_cols]
        df_emissions_activity = pd.concat([df_emissions_activity,EmissionActivityRatio])
        EmissionActivityRatio["SCENARIO"] = "Reference"
        #EmissionActivityRatio.to_excel(data['Output_location']+ "EmissionActivityRatio" + paths+".xlsx" ,index= False)
    df_emissions_activity = pd.concat([df_emissions_activity,EmissionActivityRatio])
    print ( f' Saved to {data_sheets[paths]}')
    

reading AGR at ./data/data-sheet-agriculture.xlsx
 Saved to ./data/data-sheet-agriculture.xlsx
 Saved to ./data/data-sheet-agriculture.xlsx
reading BLD at ./data/data-sheet-buildings.xlsx
 Saved to ./data/data-sheet-buildings.xlsx
reading IND at ./data/data-sheet-industry.xlsx
 Saved to ./data/data-sheet-industry.xlsx
reading TRN at ./data/data-sheet-transport.xlsx
 Saved to ./data/data-sheet-transport.xlsx
 Saved to ./data/data-sheet-transport.xlsx
reading OWN at ./data/data-sheet-ownuse.xlsx
 Saved to ./data/data-sheet-ownuse.xlsx
reading PIP at ./data/data-sheet-pipeline transport.xlsx
 Saved to ./data/data-sheet-pipeline transport.xlsx
reading NON at ./data/data-sheet-nonspecified.xlsx
 Saved to ./data/data-sheet-nonspecified.xlsx


In [341]:
#Refresh the index 

In [342]:
df_emissions_activity = df_emissions_activity.reset_index(drop = True)

In [343]:
#Find All technologies that contain ccs in name 

In [344]:
df_CCS = df_emissions_activity[df_emissions_activity["TECHNOLOGY"].str.contains('ccs')]

In [345]:
#Make a new data frame with captured emissions

In [346]:
df_captured  = df_emissions_activity.iloc[df_CCS.index]

In [347]:
#Multiply by the share of captured emissions

In [348]:
df_captured.iloc[:,7:] *= 0.8

In [349]:
df_captured.loc[:,"EMISSION"] = df_captured["EMISSION"].str.replace("CO2","CCS",regex = True )

In [350]:
df_emissions_activity.iloc[ccs.index,7:] *= 0.2

In [351]:
df_emissions_activity = pd.concat([df_emissions_activity,df_captured])

In [376]:
df_emissions_activity.to_excel(data['Output_location']+ "EmissionActivityRatio" + pd.Timestamp("today").strftime("%m-%d-%Y-%S") +".xlsx" ,index= False)